# **Segmentez des clients d'un site e-commerce**

## partie 2/4 : essais des différentes approches de modélisation

### <br> Résumé des épisodes précédents :

> &emsp; La data fournie par Olis est plutôt propre et bien organisée. Une analyse rapide révèle cependant <br>deux choses qui semblent plutôt contradictoires :<br><br> > &emsp; D'un côté, le nombre de transactions mensuelles traitées sur la période augmente régulièrement,
> tout comme le volume des ventes. En arrondissant, ils doublent, puis triplent rapidement (en moins de deux ans, cette croissance est exceptionnelle.) <br>
> Le score moyen de satisfaction des clients, > 4/5, est aussi très bon. <br><br> > &emsp; D'un autre côté... 97 % des clients n'ont jamais fait de deuxième achat. En 2 ans. <br><br> > &emsp; Etonnant, non ?? <br><br>


## 1 Importation des librairies, réglages


In [ ]:
import sys
import numpy as np
import pandas as pd
import random
from datetime import datetime

import matplotlib.pyplot as plt
from matplotlib.widgets import SpanSelector
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.colors import LinearSegmentedColormap
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score, davies_bouldin_score, adjusted_rand_score
from yellowbrick.cluster import KElbowVisualizer, SilhouetteVisualizer

from scipy.cluster.hierarchy import dendrogram, linkage, fcluster, inconsistent
from sklearn.cluster import AgglomerativeClustering, DBSCAN
from sklearn.model_selection import GridSearchCV
from sklearn.manifold import TSNE

from IPython.display import Image
# from zipfile import ZipFile

print('Python version ' + sys.version)
print('\npandas version ' + pd.__version__)
print('sns version ' + sns.__version__)

plt.style.use('ggplot')
pd.set_option('display.max_columns', 200)
sns.set(font_scale=1)


## Fonctions


In [ ]:
def quick_look(df, miss=True):
    """
    Display a quick overview of a DataFrame, including shape, head, tail, unique values, and duplicates.

    Args:
        df (pandas.DataFrame): The input DataFrame to inspect.
        check_missing (bool, optional): Whether to check and display missing values (default is True).

    The function provides a summary of the DataFrame, including its shape, the first and last rows, the count of unique values per column, and the number of duplicates.
    If `check_missing` is set to True, it also displays missing value information.
    """
    print(f'shape : {df.shape}')

    display(df.head())
    display(df.tail())

    print('uniques :')
    display(df.nunique())

    print('Doublons ? ', df.duplicated(keep='first').sum(), '\n')

    if miss:
        display(get_missing_values(df))


def lerp(a, b, t):
    """
    Linear interpolation between two values 'a' and 'b' at a parameter 't'.
    A very useful little function, used here to position annotations in plots.
    Got it coding with Radu :)

    Given two values 'a' and 'b', and a parameter 't',
    this function calculates the linear interpolation between 'a' and 'b' at 't'.

    Parameters:
    a (float or int): The start value.
    b (float or int): The end value.
    t (float): The interpolation parameter (typically in the range [0, 1], but can be outside).

    Returns:
    float or int: The interpolated value at parameter 't'.
    """
    return a + (b - a) * t


def generate_random_pastel_colors(n):
    """
    Generates a list of n random pastel colors, represented as RGBA tuples.

    Parameters:
    n (int): The number of pastel colors to generate.

    Returns:
    list: A list of RGBA tuples representing random pastel colors.

    Example:
    >>> generate_random_pastel_colors(2)
    [(0.749, 0.827, 0.886, 1.0), (0.886, 0.749, 0.827, 1.0)]
    """
    colors = []
    for _ in range(n):
        # Generate random pastels
        red = round(random.randint(150, 250) / 255.0, 3)
        green = round(random.randint(150, 250) / 255.0, 3)
        blue = round(random.randint(150, 250) / 255.0, 3)

        # Create an RGB color tuple and add it to the list
        color = (red,green,blue, 1.0)
        colors.append(color)

    return colors

print(generate_random_pastel_colors(2))


def get_missing_values(df):
    """Generates a DataFrame containing the count and proportion of missing values for each feature.

    Args:
        df (pandas.DataFrame): The input DataFrame to analyze.

    Returns:
        pandas.DataFrame: A DataFrame with columns for the feature name, count of missing values,
        count of non-missing values, proportion of missing values, and data type for each feature.
    """
    # Count the missing values for each column
    missing = df.isna().sum()

    # Calculate the percentage of missing values
    percent_missing = df.isna().mean() * 100

    # Create a DataFrame to store the results
    missings_df = pd.DataFrame({
        'column_name': df.columns,
        'missing': missing,
        'present': df.shape[0] - missing,  # Count of non-missing values
        'percent_missing': percent_missing.round(2),  # Rounded to 2 decimal places
        'type': df.dtypes
    })

    # Sort the DataFrame by the count of missing values
    missings_df.sort_values('missing', inplace=True)

    return missings_df

# with pd.option_context('display.max_rows', 1000):
#   display(get_missing_values(df))


# ma fonction d'origine (non cleanée)
def hist_distrib(dataframe, feature, bins, r, density=True):
    """
    Affiche un histogramme, pour visualiser la distribution empirique d'une variable
    Argument : df, feature num
    """
    # calcul des tendances centrales :
    mode =  str(round(dataframe[feature].mode()[0], r))
    # mode is often zero, so Check if there are non nul values in the column
    if (dataframe[feature] != 0).any():
        mode_non_nul = str(round(dataframe.loc[dataframe[feature] != 0, feature].mode()[0], r))
    else:
        mode_non_nul = "N/A"
    mediane = str(round(dataframe[feature].median(), r))
    moyenne = str(round(dataframe[feature].mean(), r))
    # dispersion :
    var_emp = str(round(dataframe[feature].var(ddof=0), r))
    coeff_var =  str(round(dataframe[feature].std(ddof=0), r)) # = écart-type empirique / moyenne
    # forme
    skewness = str(round(dataframe[feature].skew(), 2))
    kurtosis = str(round(dataframe[feature].kurtosis(), 2))

    fig, ax = plt.subplots(figsize=(12, 5))
    dataframe[feature].hist(density=density, bins=bins, ax=ax)
    yt = plt.yticks()
    y = lerp(yt[0][0], yt[0][-1], 0.8)
    t = y/20
    xt = plt.xticks()
    x = lerp(xt[0][0], xt[0][-1], 0.7)
    plt.title(feature, pad=20, fontsize=18)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    fs =13
    plt.annotate('Mode : ' + mode, xy = (x, y), fontsize = fs, xytext = (x, y), color = 'g')
    plt.annotate('Mode + : ' + mode_non_nul, xy = (x, y-t), fontsize = fs, xytext = (x, y-t), color = 'g')
    plt.annotate('Médiane : ' + mediane, xy = (x, y-2*t), fontsize = fs, xytext = (x, y-2*t), color = 'g')
    plt.annotate('Moyenne : ' + moyenne, xy = (x, y-3*t), fontsize = fs, xytext = (x, y-3*t), color = 'g')

    plt.annotate('Var emp : ' + var_emp, xy = (x, y-5*t), fontsize = fs, xytext = (x, y-5*t), color = 'g')
    plt.annotate('Coeff var : ' + coeff_var, xy = (x, y-6*t), fontsize = fs, xytext = (x, y-6*t), color = 'g')

    plt.annotate('Skewness : ' + skewness, xy = (x, y-8*t), fontsize = fs, xytext = (x, y-8*t), color = 'g')
    plt.annotate('Kurtosis : ' + kurtosis, xy = (x, y-9*t), fontsize = fs, xytext = (x, y-9*t), color = 'g')
    plt.show()

    return float(skewness) # pour eventuel passage au log

# version cleanée
def hist_distrib(dataframe, feature, bins, decimal_places, density=True):
    """
    Visualize the empirical distribution of a numerical feature using a histogram.
    Calcul des principaux indicateurs de tendance centrale, dispersion et forme.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.
        bins (int): The number of bins for the histogram.
        decimal_places (int): The number of decimal places for rounding numeric values.
        density (bool, optional): Whether to display the histogram as a density plot (default is True).

    Returns:
        float: The skewness of the feature's distribution.

    The function generates a histogram of the feature, displays various statistics, and returns the skewness of the distribution.
    """
    # Calculate central tendencies and dispersion
    mode_value = round(dataframe[feature].mode()[0], decimal_places)
    mode_non_zero = "N/A"
    if (dataframe[feature] != 0).any():
        mode_non_zero = round(dataframe.loc[dataframe[feature] != 0, feature].mode()[0], decimal_places)
    median_value = round(dataframe[feature].median(), decimal_places)
    mean_value = round(dataframe[feature].mean(), decimal_places)

    # Calculate dispersion
    var_emp = round(dataframe[feature].var(ddof=0), decimal_places)
    coeff_var = round(dataframe[feature].std(ddof=0), decimal_places)

    # Calculate shape indicators
    skewness_value = round(dataframe[feature].skew(), 2)
    kurtosis_value = round(dataframe[feature].kurtosis(), 2)

    # Create the plot
    fig, ax = plt.subplots(figsize=(12, 5))
    dataframe[feature].hist(density=density, bins=bins, ax=ax)

    # Adjust placement for annotations
    yt = plt.yticks()
    y_position = lerp(yt[0][0], yt[0][-1], 0.8)
    y_increment = y_position / 20
    xt = plt.xticks()
    x_position = lerp(xt[0][0], xt[0][-1], 0.7)

    # Add annotations with horizontal and vertical alignment
    annotation_fs = 13
    color = 'g'
    ax.annotate(f'Mode: {mode_value}', xy=(x_position, y_position), fontsize=annotation_fs,
                xytext=(x_position, y_position), color=color, ha='left', va='bottom')
    ax.annotate(f'Mode +: {mode_non_zero}', xy=(x_position, y_position - y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Median: {median_value}', xy=(x_position, y_position - 2 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 2 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Mean: {mean_value}', xy=(x_position, y_position - 3 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 3 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Var Emp: {var_emp}', xy=(x_position, y_position - 5 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 5 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Coeff Var: {coeff_var}', xy=(x_position, y_position - 6 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 6 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Skewness: {skewness_value}', xy=(x_position, y_position - 8 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 8 * y_increment), color=color, ha='left', va='bottom')
    ax.annotate(f'Kurtosis: {kurtosis_value}', xy=(x_position, y_position - 9 * y_increment), fontsize=annotation_fs,
                xytext=(x_position, y_position - 9 * y_increment), color=color, ha='left', va='bottom')

    # Label the x-axis and y-axis
    ax.set_xlabel(feature, fontsize=12)
    ax.set_ylabel('Frequency', fontsize=12)

    # Show the plot
    plt.title(f'Distribution of {feature}', pad=20, fontsize=18)
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()

    return skewness_value


def boxplot_distrib(dataframe, feature):
    """
    Affiche un boxplot, pour visualiser les tendances centrales et la dispersion d'une variable.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.

    The function generates a box plot of the feature to display central tendencies (median and mean) and dispersion.
    """
    fig, ax = plt.subplots(figsize=(10, 4))

    medianprops = {'color':"blue"}
    meanprops = {'marker':'o', 'markeredgecolor':'black',
            'markerfacecolor':'firebrick'}

    dataframe.boxplot(feature, vert=False, showfliers=False, medianprops=medianprops, patch_artist=True, showmeans=True, meanprops=meanprops)

    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


def courbe_lorenz(dataframe, feature):
    """
    Affiche une courbe de Lorenz, pour visualiser la concentration d'une variable
    Calcule l'indice de Gini
    Visualize a Lorenz curve to assess the concentration of a variable and calculate the Gini coefficient.

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to visualize.

    The function generates a Lorenz curve to assess the concentration of the feature and calculates the Gini coefficient.
    """
    fig, ax = plt.subplots(figsize=(12, 5))
    values = dataframe.loc[dataframe[feature].notna(), feature].values
    # print(values)
    n = len(values)
    lorenz = np.cumsum(np.sort(values)) / values.sum()
    lorenz = np.append([0],lorenz) # La courbe de Lorenz commence à 0

    xaxis = np.linspace(0-1/n,1+1/n,n+1)
    #Il y a un segment de taille n pour chaque individu, plus 1 segment supplémentaire d'ordonnée 0.
    # #Le premier segment commence à 0-1/n, et le dernier termine à 1+1/n.
    plt.plot(xaxis,lorenz,drawstyle='steps-post')
    plt.plot(np.arange(2),[x for x in np.arange(2)])
    # calcul de l'indice de Gini
    AUC = (lorenz.sum() -lorenz[-1]/2 -lorenz[0]/2)/n # Surface sous la courbe de Lorenz. Le premier segment (lorenz[0]) est à moitié en dessous de 0, on le coupe donc en 2, on fait de même pour le dernier segment lorenz[-1] qui est à moitié au dessus de 1.
    S = 0.5 - AUC # surface entre la première bissectrice et le courbe de Lorenz
    gini = 2*S
    plt.annotate('gini =  ' + str(round(gini, 2)), xy = (0.04, 0.88), fontsize = 13, xytext = (0.04, 0.88), color = 'g')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()


def graphs_analyse_uni(dataframe, feature, bins=50, r=5, density=True):
    """
    Affiche histogramme + boxplot + courbe de Lorenz

    Args:
        dataframe (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the numerical feature to analyze.
        bins (int, optional): The number of bins for the histogram (default is 50).
        decimal_places (int, optional): The number of decimal places for rounding numeric values (default is 5).
        density (bool, optional): Whether to display the histogram as a density plot (default is True).

    The function generates and displays an analysis of the given numerical feature, including an histogram, a box plot, and a Lorenz curve.
    """
    hist_distrib(dataframe, feature, bins, r)
    boxplot_distrib(dataframe, feature)
    courbe_lorenz(dataframe, feature)


def shape_head(df, nb_rows=5):
    """
    Affiche les dimensions et les premières lignes dùun dataframe
    Display the dimensions and the first rows of a DataFrame.

    Args:
        df (pandas.DataFrame): The input DataFrame to display.
        nb_rows (int, optional): The number of rows to display (default is 5, max is 60).

    The function prints the dimensions of the DataFrame and displays the first few rows.
    """
    print(df.shape)
    display(df.head(nb_rows))


def doughnut(df, feature, title, width=10, height=10):
    """
    Affiche la répartition d'une feature sous forme de diagramme circulaire
    Display the distribution of a feature as a doughnut chart.
    Les couleurs sont aléatoires.

    Args:
        df (pandas.DataFrame): The input DataFrame containing the feature.
        feature (str): The name of the feature to visualize.
        title (str): The title for the doughnut chart.
        width (int, optional): The width of the chart (default is 10).
        height (int, optional): The height of the chart (default is 10).

    The function creates a doughnut chart to visualize the distribution of the specified feature.
    If you don't like the colors, try running it again :)
    """
    colors = generate_random_pastel_colors(20)

    grouped_df = df.groupby(feature).size().to_frame("count_per_type").reset_index()
    pie = grouped_df.set_index(feature).copy()

    fig, ax = plt.subplots(figsize=(width, height))

    patches, texts, autotexts = plt.pie(x=pie['count_per_type'], autopct='%1.1f%%',
        startangle=-30, labels=pie.index, textprops={'fontsize':11, 'color':'#000'},
        labeldistance=1.25, pctdistance=0.85, colors=colors)

    plt.title(
    label=title,
    fontdict={"fontsize":17},
    pad=20
    )

    for text in texts:
        # text.set_fontweight('bold')
        text.set_horizontalalignment('center')

    # Customize percent labels
    for autotext in autotexts:
        autotext.set_horizontalalignment('center')
        autotext.set_fontstyle('italic')
        autotext.set_fontsize('10')

    #draw circle
    centre_circle = plt.Circle((0,0),0.7,fc='white')
    fig = plt.gcf()
    fig.gca().add_artist(centre_circle)

    plt.show()


def get_non_null_values(df):
    """
    Génère un dataframe contenant le nombre et la proportion de non-null (non-zero) valeurs pour chaque feature
    Generate a DataFrame containing the count and proportion of non-null (non-zero) values for each feature.

    Args:
        df (pandas.DataFrame): The input DataFrame to analyze.

    The function calculates and returns a DataFrame with the count and percentage of non-null (non-zero) values for each feature.
    """
    non_null_counts = df.ne(0).sum()
    percent_non_null = (non_null_counts / df.shape[0]) * 100
    non_null_values_df = pd.DataFrame({'column_name': df.columns,
                                       'non_null_count': non_null_counts,
                                       'percent_non_null': percent_non_null.round(2),
                                       'type': df.dtypes})
    non_null_values_df.sort_values('non_null_count', inplace=True)
    return non_null_values_df


def get_colors(n=7):
    """
    Generate a list of random colors from multiple colormaps.

    Args:
        n (int, optional): The number of colors to sample from each colormap (default is 7).

    Returns:
        list: A list of random colors sampled from different colormaps.
    """
    num_colors_per_colormap = n
    colormaps = [plt.cm.Pastel2, plt.cm.Set1, plt.cm.Paired]
    all_colors = []

    for colormap in colormaps:
        colors = colormap(np.linspace(0, 1, num_colors_per_colormap))
        all_colors.extend(colors)

    np.random.shuffle(all_colors)

    return all_colors


## Import data


In [ ]:
rfm_complet = pd.read_csv('data/rfm.csv', sep=',')

quick_look(rfm_complet, True)


In [ ]:
rfm = rfm_complet[['customer_unique_id', 'payment_total', 'most_recent_order', 'nb_commandes']].copy()

quick_look(rfm, True)


In [ ]:
quick_look(rfm.drop('customer_unique_id', axis=1), True)
# OK, c'est ici que le doublon apparait.

# Ca veut dire que parmis nos 95 000 clients, il y en a 2, différents, qui ont validé leurs commandes,
# du mm montant, à la mm seconde. Etonnant non ?

# fun fact
# The probability that two out of 100,000 randomly selected dates (with second precision) within the
# 63,072,000 seconds in two years will be equal is extremely low. In fact, it's highly unlikely to occur.

# Well it just did anyway.
# Does make u think. How stuff that are so highly unlikely to occur happen like all the time.


In [ ]:
# pb type

rfm['most_recent_order'] = pd.to_datetime(rfm['most_recent_order'])

plt.figure(figsize=(12, 6))
rfm['most_recent_order'].hist(bins=30, edgecolor='k')
plt.xlabel('Date')
plt.ylabel('Frequency')
plt.title('Distribution of most_recent_order')
plt.show()


## Feature engineering en vue du clustering


### Dummy model, baseline


In [ ]:
# Je me disais
# que pour commencer
# on pourait déjà étudier un clustering simple,
# voire simpliste,
# un cluster unique.

# Autrement dit, commencer par "jeter un coup d'oeil" au "client moyen" (/médian /modal?)
# Il y a des cas ou la moyenne ne signifie pas forcément grand chose,
# ou ne correspond à aucune réalité. Ici cependant, je parie qu'on peut littéralement le trouver.
# Ou presque. Challenge trouver Malcolm !

# + sérieusement,
# Ca a beau être une construction statistique, ça donne une première idée.
# Du point de vue entreprise, c'est aussi une mesure simple qui permet de faire des estimations
# (ex : un restaurant qui commande à ses fournisseurs)

# Enfin, sur le projet précédent (modèles prédictifs), on avait commencé par établir une base line
# avec un dummy model. Pour mettre en place le code simplement, et surtout avoir un point de comparaison,
# un ordre de grandeur.
# Ici je me dis que ça pourrait être utile aussi de faire comme ça du coup,
# notamment pour la troisième partie (stabilité du modèle dans le temps, maintenance)

# Note du futur : no comment...


#### Calculons Malcolm


In [ ]:
malcolm = rfm.head(1).copy()

malcolm['customer_unique_id'] = 'Malcolm'
malcolm['payment_total'] = rfm['payment_total'].mean()
malcolm['nb_commandes'] = rfm['nb_commandes'].mean()
malcolm['most_recent_order'] = rfm['most_recent_order'].mean()

display(malcolm)


In [ ]:
# Ca fait presque 8 mois que Malcolm n'a rien acheté.
# 1.034008 ça c un bon exemple de cas où le mode a plus de sens que la moyenne.
malcolm['nb_commandes'] = rfm['nb_commandes'].mode()
# 1

# Malcolm existe-t-il ?
# La date de la commande est enregistrée avec une précision d'une seconde, donc très peu probable.
# Mais on peut trouver qui est le plus proche de lui.
# Cherchons parmi les clients qui n'ont fait qu'une commande (les 97%)

subset_1_order = rfm.loc[rfm['nb_commandes'] == 1, :].copy()


#### graph classique


In [ ]:
# Matplotlib

# Create a scatter plot
plt.figure(figsize=(10, 6))
plt.scatter(subset_1_order['most_recent_order'], subset_1_order['payment_total'], s=30, c='blue', label='Data Points')
plt.xlabel('Most Recent Order Date')
plt.ylabel('Payment Total')
plt.title('2D Scatter Plot with no zooming')
plt.grid(True)

# Add Malcolm, somewhere in the middle
plt.scatter(malcolm['most_recent_order'], malcolm['payment_total'], s=100, c='red', label='M')

# Show the plot
plt.show()


#### graph interactif (zoom)


In [ ]:
# plotly (interactif)

rfm['color'] = "rgba(0, 0, 255, 0.8)"
malcolm['color'] = "rgba(255, 0, 0, 0.8)"

# Create a scatter plot with the specified color mapping
fig = px.scatter(rfm, x='most_recent_order', y='payment_total', color_discrete_sequence=['blue'])

# Customize the plot
fig.update_layout(title='Où est Charlie ?')

# Create a trace for the additional red point
red_point_trace = px.scatter(malcolm, x='most_recent_order', y='payment_total',
                             color_discrete_sequence=['red'],
                             size=[10],
                             hover_name=['Client moyen'])

# Add the red point trace to the figure
fig.add_trace(red_point_trace.data[0])

# Show the modified plot
fig.show()

# Now just zoom in !

# Au passage on voit que la data commence vraiment le 5-6 janvier 2017.
# Avant ça on a quelques dizaines (?) de commandes fin octobre 2016, et c quasiment tout :
# 2 en septembre, rien en novembre-décembre. Période de tests ?

# => On ne pourra pas entrainer nos modèles sur cette période (en partie 3)


### Format des dates


In [ ]:
# Il faut s'occuper de nos dates,
# le format pd.datetime n'étant pas exactement numérique.
# Déjà qu'on se prend des alertes juste en visualisant !

# Calculate the timestamp for the reference date
reference_date = datetime(2018, 8, 31)
reference_timestamp = reference_date.timestamp()

# Convert the 'most_recent_order' datetime feature into a timestamp
rfm['most_recent_order_timestamp'] = rfm['most_recent_order'].apply(lambda x: x.timestamp())

# Optionel, pour lire + facilement :
rfm['recent_timestamp'] = reference_timestamp - rfm['most_recent_order_timestamp']
rfm['recent_timedelta'] = reference_date - rfm['most_recent_order'] # type timedelta
rfm['recent_datetime'] = reference_date - rfm['recent_timedelta']

quick_look(rfm)

# Attention, 'recent_datetime' se lit "dans l'autre sens" par rapport aux autres :
# = 0 pour la date de référence, augmente qd on remonte vers la passé
# max value = commande la + ancienne
# (voir histogramme bleu ci-dessous)


In [ ]:
# Check everything's ok

fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot a histogram for most_recent_order_timestamp on the first subplot
axes[0].hist(rfm['most_recent_order'], bins=20, color='skyblue', alpha=0.8)
axes[0].set_xlabel('most_recent_order')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of most_recent_order (datetime)')

# Plot a histogram for recent on the second subplot
axes[1].hist(rfm['most_recent_order_timestamp'], bins=20, color='skyblue', alpha=0.8)
axes[1].set_xlabel('most_recent_order_timestamp')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of most_recent_order_timestamp')

plt.tight_layout()  # Ensure proper spacing between subplots
plt.show()


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot a histogram for most_recent_order_timestamp on the first subplot
axes[0].hist(rfm['recent_timestamp'], bins=20, color='lightgreen', alpha=0.8)
axes[0].set_xlabel('recent_timestamp')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribution of recent_timestamp')

# Plot a histogram for most_recent_order on the second subplot
axes[1].hist(rfm['recent_datetime'], bins=20, color='skyblue', alpha=0.8)
axes[1].set_xlabel('recent_datetime')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Distribution of recent_datetime')

plt.tight_layout()
plt.show()

# Nickel

### Visualisation 3D (toute la data)


In [ ]:
# Create a 3D scatter plot
fig = px.scatter_3d(rfm, x='recent_datetime', y='nb_commandes', z='payment_total')

# Customize the plot
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='Our data in 3D')

fig.update_layout(
    width=800,  # Specify the width in pixels
    height=600  # Specify the height in pixels
)

fig.show()

# On voit à nouveau que notre donnée est très compacte sur les 2 axes correspondant
# aux deux features très asymétriques.

# Pas sûr que ça pose vraiment problème aux modèles, mais nous on y verra difficilement
# si tous les points sont les uns sur les autres.


### Passage au log (skewed features)


In [ ]:
# On va préparer un second jeu de données où les 2 features asym ont été passées au log

# R ('recent') n'est pas très asymétrique (skewness = 0.45),
# en revanche F et M le sont

features_RFM = ['recent_timestamp', 'nb_commandes', 'payment_total'] # = R F M
rfm_log = rfm[features_RFM].copy()

# valeurs min < 0, mais > -1
rfm_log['nb_commandes'] = np.log(rfm['nb_commandes'] + 1)
rfm_log['payment_total'] = np.log(rfm['payment_total'] + 1)

graphs_analyse_uni(rfm_log, 'nb_commandes', bins=50, r=2, density=True) # F
graphs_analyse_uni(rfm_log, 'payment_total', bins=50, r=2, density=True) # M


In [ ]:
# Ok c déjà + espacé et symétrique
# En tous cas pour le montant
# Moins évident pour la fréquence, qui prend des valeurs discrètes

fig = px.scatter_3d(rfm_log, x='recent_timestamp', y='nb_commandes', z='payment_total')

# Customize the plot
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='Our data in 3D')

fig.update_layout(
    width=800,  # Specify the width in pixels
    height=600  # Specify the height in pixels
)

fig.show()

# + aéré


### Scaling


In [ ]:
# On peut scale

# Initialize the StandardScaler
scaler = StandardScaler()

# Fit and transform the selected columns
rfm_scaled = rfm[features_RFM].copy()
rfm_scaled[features_RFM] = scaler.fit_transform(rfm[features_RFM])

quick_look(rfm_scaled)
rfm_scaled.describe()

# Do the same for log dataset
rfm_log_scaled = rfm_log[features_RFM].copy()
rfm_log_scaled[features_RFM] = scaler.fit_transform(rfm_log[features_RFM])

quick_look(rfm_log_scaled)
rfm_log_scaled.describe()

# Juste le tps de regarder à quoi ressemblent nos nouvelles distribs,
# et on est prêt pour notre premier k-means


In [ ]:
# Observons nos nouvelles distribs

graphs_analyse_uni(rfm_scaled, 'recent_timestamp', bins=50, r=2, density=True)
# graphs_analyse_uni(rfm_log_scaled, 'recent', bins=50, r=2, density=True)
# (c la mm chose, on n'a pas log cette feature)

# Lorenz ???


In [ ]:
graphs_analyse_uni(rfm_scaled, 'nb_commandes', bins=50, r=2, density=True) # F
graphs_analyse_uni(rfm_log_scaled, 'nb_commandes', bins=50, r=2, density=True) # F

# Gini WTF ??


In [ ]:
graphs_analyse_uni(rfm_scaled, 'payment_total', bins=50, r=2, density=True) # M
graphs_analyse_uni(rfm_log_scaled, 'payment_total', bins=50, r=2, density=True) # M

# J'avais jamais vu des courbes de Lorenz comme ça
# Je devrais peut-être aller dormir ^^


### Partitionnement manuel


In [ ]:
# Les moyennes sont bien à zero, et les variances à 1.

# Le problème de distributions très asymétriques semble corrigé par le passage au log.
# A vérifier pour nb_commandes > 1

# Il est possissible qu'on soit amené à encore étudier séparément
# les 97% de clients qui n'ont fait qu'une commande (en 2D du coup),
# et les 3% de clients "réguliers" (nb_commandes >= 2) (en 3D).

# On a déjà créé un subset pour les 97%, mais on a modifié les dates depuis.
subset_1_order  = rfm.loc[rfm['nb_commandes'] == 1, features_RFM].copy()

# Pour les 3% :
subset_several_orders = rfm.loc[rfm['nb_commandes'] > 1, features_RFM].copy()

# log
subset_1_order_log = subset_1_order[features_RFM].copy()
subset_1_order_log['nb_commandes'] = np.log(subset_1_order['nb_commandes'] + 1)
subset_1_order_log['payment_total'] = np.log(subset_1_order['payment_total'] + 1)

subset_several_orders_log = subset_several_orders.copy() # On a déjà sélectionné les features
subset_several_orders_log['nb_commandes'] = np.log(subset_several_orders['nb_commandes'] + 1)
subset_several_orders_log['payment_total'] = np.log(subset_several_orders['payment_total'] + 1)

# scaling
subset_1_order_scaled = subset_1_order[features_RFM].copy()
subset_1_order_scaled[features_RFM] = scaler.fit_transform(subset_1_order[features_RFM])

subset_several_orders_scaled = subset_several_orders.copy()
subset_several_orders_scaled[features_RFM] = scaler.fit_transform(subset_several_orders[features_RFM])

# log + scaling
subset_1_order_log_scaled = subset_1_order_log[features_RFM].copy()
subset_1_order_log_scaled[features_RFM] = scaler.fit_transform(subset_1_order_log[features_RFM])

subset_several_orders_log_scaled = subset_several_orders_log[features_RFM].copy()
subset_several_orders_log_scaled[features_RFM] = scaler.fit_transform(subset_several_orders_log[features_RFM])

# Encore une fausse bonne idée (Captain Hinsight)


## 2 k-means


### les scores que nous allons utiliser pour évaluer la qualité de nos clusters


In [ ]:
# On commence 2 par 2 en 2D,
# où on essaie direct les 3 ensemble ?

# On va tenter la 3D pour avoir directement une vue d'ensemble.

# Un petit mot sur nos "scores" ?

# L'inertie (moyenne des carrés des distances entre points et centroide)
# est le plus rapide à calculer.
# Encore que en pratique, c serré avec tightness. et avec Davies-Bouldin.Bbref.
# L'inertie nous donne une idée sur la dispersion des points dans le cluster, par rapport au centroide,
# sur la variance intracluster
# Méthode du coude

# Assez similaire, le coeff de Tightness, un peu plus long à calculer, nous dit en + si notre cluster
# est compact ou pas, si tous les points sont plutôt resserrés ou distants les uns des autres, en moyenne.
# + petit = mieux pour nous, bon cluster !

# En + de 2 ou 3 dimensions, je me dis que l'inertie commence à avoir plus de sens car
# les distances vont exploser, mais la variance pas forcément.
# Du coup un score + stable ?

# Tightness est aussi le seul score sans raccourci ds sckikit.
# G l'impression qu'inertia est + utilisé.

# La Separation nous dit si nos clusters sont (bien) éloignés.
# Ca aussi on préfère :)
# Combinée aux Tightness des différents clusters, elle forme le score de Davies_Bouldin,
# DB = (T moyen) / (S moyen)

# Enfin l'indicateur le plus utilisé,
# et malheureusement, le + long à calculer bien sûr :)
# Le silhouette score, qui vérifie si chaque point est placé dans le "bon" cluster
# = difference entre (moyenne distances entre point et autres points de son cluster)
# et (idem cluster voisin (le plus proche))
# normalisée par le + gd des 2 -> résultat compris dans [-1, 1]
# ca c pour 1 point, le silhouette score en pratique c la moyenne de ca pour ts les points.


### Nombre optimal de clusters, using different scores


#### inertia


In [ ]:
# Plot the score values for different numbers of clusters
def plot_score(score_list, score_name, max=15):
    """
    Plot the score values for different numbers of clusters.

    Args:
        score_list (list): A list of score values for different numbers of clusters.
        score_name (str): The name of the score being plotted (e.g., 'Silhouette Score').
        max (int, optional): The maximum number of clusters to plot (default is 15).

    This function generates a line plot to visualize how a specific clustering score varies with different numbers of clusters (k).
    It helps in identifying the optimal number of clusters using the given clustering score.

    Parameters:
    - score_list: A list of score values for different values of k.
    - score_name: The name of the score to display in the plot (e.g., 'Silhouette Score', 'Inertia', etc.).
    - max: The maximum number of clusters to plot (default is 15).
    """
    plt.figure(figsize=(8, 6))
    plt.plot(range(2, max), score_list, marker='o', linestyle='-', color='b')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel(score_name)
    plt.title('Optimal k, using ' + score_name + ' score')
    plt.grid(True)
    plt.show()


# Calculate Tightness
def compute_tightness(data, cluster_labels, cluster_centers):
    """
    Calculate the tightness score for a clustering.

    Args:
        data (numpy.ndarray): The data points.
        cluster_labels (numpy.ndarray): Labels indicating the cluster assignment for each data point.
        cluster_centers (numpy.ndarray): The centers of the clusters.

    Returns:
        float: The tightness score, a measure of how close data points are to their cluster centers.

    The tightness score quantifies the compactness of clusters in a clustering. It is computed as the average distance of data points within each cluster to their respective cluster center. A lower tightness score indicates that data points are closer to their cluster centers, indicating tighter clusters.

    Parameters:
    - data: The data points in the dataset.
    - cluster_labels: An array of cluster labels for each data point.
    - cluster_centers: The centers of the clusters.
    """
    tightness_score = 0
    for i in range(len(cluster_centers)):
        cluster_points = data[cluster_labels == i]
        center = cluster_centers[i]
        distances = np.linalg.norm(cluster_points - center, axis=1)
        tightness_score += np.mean(distances)
    tightness_score /= len(cluster_centers)
    return tightness_score


def find_best_nb_clusters_for_kmeans(df=rfm_scaled, features=features_RFM, scores=[], max=15):
    """
    Find the best number of clusters for K-Means clustering based on various scoring methods.

    Args:
        df (pd.DataFrame): The dataset for clustering.
        features (list): List of feature columns for clustering.
        scores (list): List of scoring methods to use ('inertia', 'tightness', 'db', 'silhouette').
        max (int): The maximum number of clusters to consider (default is 15).

    This function performs K-Means clustering with different numbers of clusters (k) and evaluates the clustering results using various scoring methods, including inertia, tightness, Davies-Bouldin index, and silhouette score. It then plots the scores to help determine the optimal number of clusters based on the chosen scoring methods.

    Parameters:
    - df: The dataset for clustering.
    - features: List of feature columns for clustering.
    - scores: List of scoring methods to use ('inertia', 'tightness', 'db', 'silhouette').
    - max: The maximum number of clusters to consider (default is 15).
    """
    # Create empty lists to store scores for different k values
    inertia_scores = []
    tightness_scores = []
    davies_bouldin_scores = []
    silhouette_scores = []

    # Test different numbers of clusters from 2 to 15
    for k in range(2, max):
        kmeans = KMeans(n_clusters=k, n_init=10, random_state=42)
        kmeans.fit(df[features])

        if 'inertia' in scores:
            inertia_scores.append(kmeans.inertia_)
        if 'tightness' in scores:
            tightness = compute_tightness(df[features], kmeans.labels_, kmeans.cluster_centers_)
            tightness_scores.append(tightness)
        if 'db' in scores:
            davies_bouldin = davies_bouldin_score(df[features], kmeans.labels_)
            davies_bouldin_scores.append(davies_bouldin)
        if 'silhouette' in scores:
            silhouette = silhouette_score(df[features], kmeans.labels_)
            silhouette_scores.append(silhouette)

    if 'inertia' in scores:
        plot_score(inertia_scores, 'inertia', max=max)
    if 'tightness' in scores:
        plot_score(tightness_scores, 'tightness', max=max)
    if 'db' in scores:
        plot_score(davies_bouldin_scores, 'Davies-Bouldin', max=max)
    if 'silhouette' in scores:
        plot_score(silhouette_scores, 'silhouette', max=max)


find_best_nb_clusters_for_kmeans(scores=['inertia'], max=15)

# best nb de cluster : 4 (elbow method)
# tps : 5-6s


#### tightness


In [ ]:
find_best_nb_clusters_for_kmeans(scores=['tightness'], max=15)

# best nb de cluster : 5 (min)
# tps : 5-6s


#### Davies-Bouldin


In [ ]:
find_best_nb_clusters_for_kmeans(df=rfm_scaled, scores=['db'], max=15)

# best nb de cluster : 4 (min)
# tps : 5-6s


#### Subset, sampling


In [ ]:
# Les temps de calcul vont augmenter très vite pour silhouette. On va avoir besoin de subsets
# (Calculer seulement l'inertie prend 5 seconds. Avec les 3 scores, il faut une heure...)

# Les résultats sont concordants (ici, 4 clusters, 5 si on privilégie tightness)

# utiliser seulement l'inertie ?
# -> Subsampling ?
# Parallel Processing ?

rfm_scaled_tenth = rfm_scaled[::10].copy()

# Vérifions que la forme du subset est la même
# on a selectionné 1 ligne ttes les 10 lignes, donc c une selection aléatoire,
# tant que les données de départ ne sont pas ordonnées (modulo 10).

fig = px.scatter_3d(rfm_scaled, x='recent_timestamp', y='nb_commandes', z='payment_total')
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='Our data in 3D')
fig.update_layout(
    width=600,  # Specify the width in pixels
    height=400  # Specify the height in pixels
)
fig.show()

fig = px.scatter_3d(rfm_scaled_tenth, x='recent_timestamp', y='nb_commandes', z='payment_total')
fig.update_traces(marker=dict(size=5))
fig.update_layout(title='Our data in 3D')
fig.update_layout(
    width=600,  # Specify the width in pixels
    height=400  # Specify the height in pixels
)
fig.show()

# Ajuster ?
# Ca a l'air pas mal


In [ ]:
fig1 = go.Figure()

scatter1 = go.Scatter3d(x=rfm_scaled['recent_timestamp'], y=rfm_scaled['nb_commandes'], z=rfm_scaled['payment_total'], mode='markers', marker=dict(size=5))
fig1.add_trace(scatter1)
fig1.update_layout(title='Our data in 3D - Subset 1')
fig1.update_layout(width=400, height=300)

# Create the second scatter plot
fig2 = go.Figure()

scatter2 = go.Scatter3d(x=rfm_scaled_tenth['recent_timestamp'], y=rfm_scaled_tenth['nb_commandes'], z=rfm_scaled_tenth['payment_total'], mode='markers', marker=dict(size=5))
fig2.add_trace(scatter2)
fig2.update_layout(title='Our data in 3D - Subset 2')
fig2.update_layout(width=400, height=300)

# Combine the two figures
fig = go.Figure(data=[fig1.data[0], fig2.data[0]])
fig.show()

# Je crois qu'on voyait mieux séparés


#### Silhouette


In [ ]:
# On va donc utiliser ce subset pour aller + vite

# find_best_nb_clusters_for_kmeans(df=rfm_scaled_tenth, scores=['silhouette'], max=15)

# best nb de cluster : 4 (max)
# tps : environ 1 min, sur 1/10 des datas

# Mm resultat sur dataset complet, mais bcp + long, environ 45 min
# find_best_nb_clusters_for_kmeans(df=rfm_scaled, scores=['silhouette'], max=15)
Image(filename='img/silhouette_rfm.png')


### Yellowbrick


In [ ]:
# Instantiate the clustering model and visualizer
model = KMeans(n_init='auto')
visualizer = KElbowVisualizer(model, k=(2,12))

visualizer.fit(rfm_scaled)    # Fit the data to the visualizer
visualizer.poof()    # Draw/show/poof the data

# bcp, bcp + rapide que notre méthode (silhouette_score, from sklearn.metrics)
# Comment ils font ?


In [ ]:
# Instantiate the clustering model and visualizer
model = KMeans(n_init='auto')
visualizer = KElbowVisualizer(model, k=(2,12))

visualizer.fit(rfm_scaled)    # Fit the data to the visualizer
visualizer.poof()    # Draw/show/poof the data

# Comment c si rapide ???


In [ ]:
# Instantiate the clustering model and visualizer
model = KMeans(n_init='auto')
visualizer = KElbowVisualizer(model, k=(2,12), metric='calinski_harabasz', timings=False)

visualizer.fit(rfm_scaled)    # Fit the data to the visualizer
visualizer.poof()    # Draw/show/poof the data

# 3-4


In [ ]:
model = KMeans(4, n_init='auto')
visualizer = SilhouetteVisualizer(model)

# visualizer.fit(rfm_scaled)    # Fit the data to the visualizer
# visualizer.poof()    # Draw/show/poof the data

# + long
# 2 gros clusters, 2 bcp - étendus

Image(filename='img/poof.png')


### Stabilité


In [ ]:
# Etude de la stabilité des clusters sur une sous-partition aléatoire
# methodo pas bon là
# Pourquoi j'ai codé ça ??

# Il faut verif si j'ai les mm clusters, pas si le nb reste le mm sur une ss partit aleatoire !

# Remarque, c tjs une bonne nouvelle que le clustering semble stable sur de "petites" partitions
# au 1/20 ième : cela correspond environ à 1 mois de data pour la partie 3.

# Par contre, encore une fois, pas certain que ce soit encore le cas en 4 ou 5 dimensions.
# A chaque dimension ajoutée, toute la data est étirée le long d'un nouvel axe,
# ce qui crée beaucoup de vide.
# Du coup il faudra peut-être bcp + de points pour qu'un subset construit par sample ait des chances de
# vaguement reproduire la forme globale. A tester

n_iter = 10

for i in range(n_iter):
    # Shuffle the DataFrame randomly
    shuffled_df = rfm_scaled.sample(frac=1, random_state=i)  # frac=1 shuffles all rows

    # Sample a portion of the shuffled DataFrame
    sample_size = 5000  # Adjust this to your desired sample size
    sampled_df = shuffled_df.head(sample_size)

    inertia = []

    for k in range(2, 11):
        kmeans = KMeans(n_clusters=k, n_init=10, random_state=i)
        kmeans.fit(shuffled_df[features_RFM])

        inertia.append(kmeans.inertia_)

    print(inertia)

    # Plot the inertia values for different numbers of clusters
    plt.figure(figsize=(8, 6))
    plt.plot(range(2, 11), inertia, marker='o', linestyle='-', color='b')
    plt.xlabel('Number of Clusters (k)')
    plt.ylabel('Inertia')
    plt.title('Elbow Method for Optimal k')
    plt.grid(True)

# Très stable
# Vérifier mieux que les clusters sont proches, similaires : -> utiliser l'ARI ?


### ARI


In [ ]:
# Nous allons avoir besoin d'une fonction qui retourne les labels assignés par le k-means
# (pour visualisation et ARI).

# Assuming 4 is the optimal number of clusters
def assign_labels_using_kmeans(alea, df=rfm_scaled, features=features_RFM, best_k=4, silhouette=False):
    """
    Assign cluster labels to data points using K-Means clustering.

    Args:
        alea (int): Random seed for reproducibility.
        df (pd.DataFrame): The dataset for clustering.
        features (list): List of feature columns for clustering.
        best_k (int): The number of clusters for K-Means (default is 4).
        silhouette (bool): Whether to print the silhouette score for the K-Means clustering (default is False).

    This function performs K-Means clustering on the given data with a specified number of clusters (best_k) and assigns cluster labels to data points. Optionally, it can print the silhouette score as a measure of clustering quality.

    Returns:
    - labels (array): Cluster labels assigned to data points.
    """
    kmeans = KMeans(n_clusters=best_k, n_init='auto', random_state=alea)
    kmeans.fit(df[features])

    if silhouette == True:
        print('silhouette_score for kmeans = ', silhouette_score(df[features], kmeans.labels_))

    return kmeans.labels_

# L'ARI est souvent utilisé pour comparer le clustering obtenu par un modèle à une partition connue,
# pour tester la qualité du modèle. Ici c'est un peu différent,
# Nous n'avons pas de partition pré-établie. Nous allons seulement tester la stabilité,
# pas la qualité, en comparant les labels obtenus successivement à ceux de l'essai précédent.

labels_kmeans = []

for i in range(0, 15):
    labels = assign_labels_using_kmeans(alea=i)
    labels_kmeans.append(labels)
    if i > 0:
        # Calculate ARI
        ari = adjusted_rand_score(labels_kmeans[i], labels_kmeans[i-1])
        print(f'ARI = {ari}', '\n')

display(labels_kmeans)


### Visualisation, premier clustering RFM


#### wrong legend


In [ ]:
# Ca a l'air super stable.
# Visualisons !

def plot_kmeans_3D(df=rfm_scaled, legend=False):
    # Create a list of colors based on labels
    cluster_colors = px.colors.qualitative.Set2[:len(set(labels_kmeans[0]))]

    fig = px.scatter_3d(df, x='recent_timestamp', y='nb_commandes', z='payment_total', color=labels_kmeans[0],
                        color_discrete_sequence=cluster_colors, opacity=0.8)

    fig.update_traces(marker=dict(size=5), showlegend=False)
    fig.update_layout(title='K-Means Clustering in 3D')

    fig.update_layout(
        width=800,  # Specify the width in pixels
        height=600  # Specify the height in pixels
    )
    fig.show()

plot_kmeans_3D()

# la légende est displayed qd mm ?


#### legend ok for now, wrong colors


In [ ]:
# La légende sert pas à gd chose ici, c + clair sans.
# Ou en mode discret (catégorique)

def plot_kmeans_3D(df=rfm_scaled, legend=True):
    # Convert cluster labels to integer and then to string format
    cluster_labels_str = labels_kmeans[0].astype(int).astype(str)
    # Create a list of colors based on labels
    cluster_colors = px.colors.qualitative.Set1[:len(set(labels_kmeans[0]))]

    # Create a 3D scatter plot with custom legend
    fig = px.scatter_3d(df, x='recent_timestamp', y='nb_commandes', z='payment_total', color=cluster_labels_str,
                        color_discrete_sequence=cluster_colors, opacity=0.7)

    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='K-Means Clustering in 3D')

    fig.update_layout(
        width=800,  # Specify the width in pixels
        height=600  # Specify the height in pixels
    )
    fig.show()

plot_kmeans_3D(legend=True)


#### more color tests


In [ ]:
# tests couleurs

custom_color_set_4 = ['red', 'pink', 'lightgreen', 'yellow']

# Define a function to plot K-Means clusters in 3D with a custom color set
def plot_kmeans_3D(df=rfm_scaled, legend=True, color_set=custom_color_set_4, opacity=1.0):
    """
    Visualize K-Means clustering in 3D using Plotly Express.

    Args:
        df (DataFrame): The DataFrame containing the data.
        legend (bool): Whether to display the legend (default is False).

    This function creates a 3D scatter plot of K-Means clustering results using Plotly Express. It visualizes the data points based on their coordinates in three dimensions (recent_timestamp, nb_commandes, payment_total) and colors them according to the cluster labels.

    Parameters:
    - df (DataFrame): The input DataFrame.
    - legend (bool): Set to True to display the legend.

    Returns:
    - None
    """
    # Convert cluster labels to integer and then to string format
    cluster_labels_str = labels_kmeans[0].astype(int).astype(str)
    # Create a list of colors based on labels
    if color_set is None:
        cluster_colors = custom_color_set_4[:len(set(labels_kmeans[0]))]
    else:
        cluster_colors = color_set[:len(set(labels_kmeans[0]))]

    # Create a 3D scatter plot with custom legend
    fig = px.scatter_3d(df, x='recent_timestamp', y='nb_commandes', z='payment_total', color=cluster_labels_str,
                        color_discrete_sequence=cluster_colors, opacity=opacity)

    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='K-Means Clustering in 3D')

    fig.update_layout(
        width=800,  # Specify the width in pixels
        height=600  # Specify the height in pixels
    )
    fig.show()

# Example usage with a different color set
plot_kmeans_3D(color_set=px.colors.qualitative.Set3)
# plot_kmeans_3D(color_set=px.colors.qualitative.Prism)

list_other_color_sets = [px.colors.qualitative.Set3,
                         px.colors.qualitative.Dark24,
                         px.colors.qualitative.Pastel1,
                         px.colors.qualitative.Pastel2,
                         px.colors.qualitative.Prism,
                         px.colors.qualitative.Vivid,
                         px.colors.qualitative.Safe]

for color_set in list_other_color_sets:
    print(f"Using color set: {color_set}")
    # plot_kmeans_3D(color_set=color_set)

# ! OK, Pas abuser de plotly ds la mm cell !


In [ ]:
plot_kmeans_3D(color_set=px.colors.qualitative.Pastel1)
# plot_kmeans_3D(color_set=px.colors.qualitative.Pastel2)

# J'aime bcp les pastels, mais c pas le + lisible


In [ ]:
# plot_kmeans_3D(color_set=px.colors.qualitative.Vivid)
# plot_kmeans_3D(color_set=px.colors.qualitative.Safe)

# Décommentez à vos risques et périls. C très moche^^


#### custom colormap


In [ ]:
# Au final, jamais mieux servi...
plot_kmeans_3D()


### Résultat, interprétation


In [ ]:
# On peut déjà interpréter les clusters obtenus, cependant nos données de base sont très compactes,
# on verra peut-être mieux en utilisant le jeu transformé.

plot_kmeans_3D(df=rfm_log_scaled)

# On voit mieux, merci la fonction log !

# En jaune (cluster 3), les clients qui ont le plus dépensé au total
# en vert (cluster 2) les clients réguliers
# en rouge (cluster 0) les nveaux clients (1 petit achat récent)
# en rose (cluster 1) les pires clients : 1 petit achat il y a lgtmps.

# C'est un bon début : le modèle est stable, les clusters ont du sens du point de vue métier.
# Cependant Olis attend une analyse + précise


## 3 dendro


### sampling obligatoire


In [ ]:
# Work on a small sample (fastest but very distorted data)
# dendro_df = rfm_log_scaled[features_RFM][::50].copy()

# Use complete data : impossible without modifying notebook params
# (dataset too big for an algo with this order of complexity)
# dendro_df = rfm_log_scaled[features_RFM].copy()

# Good compromise, on a déjà vérifié que la topologie des datas est conservée assez fidèlement
dendro_df = rfm_log_scaled[::10].copy()

# Perform hierarchical clustering
linkage_matrix = linkage(dendro_df, method='ward')  # You can choose different linkage methods

# Create a dendrogram
dendrogram(linkage_matrix, labels=dendro_df.index.values)

# Display the dendrogram
plt.title("Dendrogram")
plt.xlabel("Sample Index")
plt.ylabel("Distance")

plt.axhline(y=82, color='orange', linestyle='--', label='Threshold')
# best number of clusters = 4 with this first "method"
# (More like a quick empirical "rule of thumb")

plt.show()

# Bcp + long que k-means !

# Sur [::10]
display(Image(filename='img/dendro_rfm.png'))


### nb optimal de cluster, using inconsistency


In [ ]:
# Determine the optimal number of clusters using inconsistency
depth = 5  # You can adjust this depth parameter
inconsistencies = inconsistent(linkage_matrix, depth)
optimal_cluster_count = inconsistencies.argmax() + 1

# Plot the inconsistency values
plt.figure()
plt.plot(range(2, len(inconsistencies) + 2), inconsistencies)
plt.title("Inconsistency vs. Number of Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Inconsistency")
plt.axvline(x=optimal_cluster_count, color='red', linestyle='--', label='Optimal Cluster Count')
plt.legend()
plt.show()


In [ ]:
# 38 000 clusters, c peut-être mathématiquement optimal, mais...
# Bon courage pour l'interprétation !

# Plot the inconsistency values for up to 15 clusters
max_clusters_to_plot = 15
optimal_cluster_count = inconsistencies[:max_clusters_to_plot - 1].argmax() + 2 # index starts at 2

plt.figure()

plt.plot(range(2, max_clusters_to_plot + 2), inconsistencies[:max_clusters_to_plot])

plt.title("Inconsistency vs. Number of Clusters")
plt.xlabel("Number of Clusters")
plt.ylabel("Inconsistency")
plt.axvline(x=optimal_cluster_count, color='red', linestyle='--', label='Optimal Cluster Count')
plt.legend()
plt.show()

# On retrouve nos 4 clusters
# or do we ?

optimal_cluster_count = 4

# Extract cluster labels
cluster_labels_dendro_4 = fcluster(linkage_matrix, t=optimal_cluster_count, criterion='maxclust')

# Compare dendro clusters to kmeans clusters
labels_kmeans4_tenth = assign_labels_using_kmeans(alea=42, df=dendro_df, features=features_RFM, best_k=4, silhouette=True)
ari = adjusted_rand_score(labels_kmeans4_tenth, cluster_labels_dendro_4)
print(f'ARI = {ari}', '\n')

# J'avais oublié qu'on travaillait d'abord sur un petit subset pour gagner du tps...
# Décommenter l'ARI ci-dessus seulement sur jeu sliced au 1/10 !
# C bon, c corrigé,

# Silhouette kmeans = 0.37
# ARI = 0.6, les 2 partitionnements semblent assez différents


### Qualité des clusters


In [ ]:
# Enfin, évaluons la qualité des clusters obtenus

def evaluate_dendro_clusters(linkage='ward'):
    """
    Evaluate the optimal number of clusters using Agglomerative Clustering and Silhouette Score.

    Args:
        linkage (str): The linkage method for Agglomerative Clustering (default is 'ward').

    This function performs Agglomerative Clustering with varying numbers of clusters and calculates the Silhouette Score for each clustering. It then plots the Silhouette Score against the number of clusters and identifies the optimal number of clusters based on the highest Silhouette Score.

    Parameters:
    - linkage (str): The linkage method for Agglomerative Clustering.

    Returns:
    - None
    """
    silhouette_scores = []

    for n_clusters in range(2, 10):
        # Create an AgglomerativeClustering model with the chosen linkage method and the current number of clusters
        model = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)

        # Fit the model to your data
        cluster_labels = model.fit_predict(dendro_df)

        # Calculate the silhouette score for the current clustering
        silhouette_avg = silhouette_score(dendro_df, cluster_labels)
        silhouette_scores.append(silhouette_avg)

    # Plot the silhouette scores
    plt.figure()
    plt.plot(range(2, 10), silhouette_scores, marker='o', linestyle='-', color='b')
    plt.title("Silhouette Score vs. Number of Clusters")
    plt.xlabel("Number of Clusters")
    plt.ylabel("Silhouette Score")
    plt.grid(True)

    # Find the optimal number of clusters (highest silhouette score)
    optimal_clusters = silhouette_scores.index(max(silhouette_scores)) + 2
    plt.axvline(x=optimal_clusters, color='r', linestyle='--', label=f'Optimal Clusters ({optimal_clusters})')
    plt.legend()

    plt.show()


evaluate_dendro_clusters()

# Pour nb clusters > 2, on retrouve encore 4 comme meilleure valeur.
# Le silhouette score du kmeans était meilleur.
# Tant mieux, l'algo le + rapide est aussi le + performant ici.


In [ ]:
# Testons d'autres méthodes

evaluate_dendro_clusters(linkage='single')

# ??


In [ ]:
evaluate_dendro_clusters(linkage='complete')

# le silhouette score est curieux ici aussi


In [ ]:
evaluate_dendro_clusters(linkage='average')

# c quoi ces silhouettes ??


### Conclusion


In [ ]:
# Pour l'instant le kmeans l'emporte,

# - pour la qualité supérieure de son clustering, comme le montre le silhouette score
# (particulièrement significatif, je trouve, qd on compare le clustering du kmeans
# au clustering hiérarchique utilisant la méthode de ward. En effet, tous deux cherchent à minimiser
# la variance intracluster.)

# - pour sa capacité à travailler sur le jeu de données entier, rapidement.
# En effet la "limitation" du clustering hiérarchique (sa complexité algorithmique lourde)
# sera encore plus accentuée à chaque ajout de feature.

# Notons cependant que pour l'étape finale, le modèle choisi ne travaillera plus que sur une tranche
# réduite du dataset, mise à jour régulièrement. Le clustering hiérarchique reste donc une possibilité
# pour l'instant.


## 4 DBSCAN


In [ ]:
# "à la main"

def plot_DBSCAN_3D(df=rfm_log_scaled, legend=True, color_set=custom_color_set_4, opacity=1.0):
    # Specify the DBSCAN parameters (eps and min_samples)
    eps = 0.2  # Maximum distance to form a dense cluster
    min_samples = 5  # Minimum number of samples in a neighborhood to be considered a core point

    # Initialize the DBSCAN clustering model
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)

    # Fit the DBSCAN model to the data
    dbscan.fit(df)

    # Extract cluster labels (-1 represents noise points)
    labels = dbscan.labels_
    print(f'DBSCAN propose {len(set(labels))} clusters.')

    # Convert cluster labels to integer and then to string format
    cluster_labels_str = labels.astype(int).astype(str)

    # Create a list of colors based on labels
    if color_set is None:
        cluster_colors = custom_color_set_4[:len(set(labels))]
    else:
        cluster_colors = color_set[:len(set(labels))]

    # Create a 3D scatter plot with custom legend
    fig = px.scatter_3d(df, x='recent_timestamp', y='nb_commandes', z='payment_total', color=cluster_labels_str,
                        color_discrete_sequence=cluster_colors, opacity=opacity)

    fig.update_traces(marker=dict(size=5), showlegend=legend)
    fig.update_layout(title='DBSCAN Clustering in 3D')

    fig.update_layout(
        width=800,  # Specify the width in pixels
        height=600  # Specify the height in pixels
    )
    fig.show()


plot_DBSCAN_3D()

# mm ?

# On passe très vide de 1 cluster à une multide de clusters,
# pour des valeurs de eps assez proches.
# Difficile d'obtenir juste qq clusters qui aient du sens métier.

# J'ai l'impression que nos points sont espacés de manière trop régulière pour le DBSCAN,
# la topologie n'est pas intéressante pour cet algo car
# Il n'y a pas de zone de vide qui sépare naturellement nos données.

# Ce ne sera cependant plus forcément le cas en dimensions supérieures, donc
# DBSCAN à réessayer qd on aura ajouté des features.


In [ ]:
# Avec un genre de gridsearchcv

# Turns out, GridSearchCV has no score really suitable for clustering evaluation
# We need to do a "manual gridsearch"

# Define the range of eps and min_samples values
eps_values = [0.1, 0.2, 0.3]
min_samples_values = [3, 5, 7]

# Initialize lists to store scores and cluster counts
dbscan_scores = []
cluster_counts = []

# Perform manual grid search
for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(rfm_log_scaled)
        # Silhouette prend bcp + de tps
        # score = silhouette_score(rfm_log_scaled, labels)
        # Utilisons D-B ici :
        score = davies_bouldin_score(rfm_log_scaled[features_RFM], labels)
        dbscan_scores.append(score)
        nb_clusters = len(set(labels))
        cluster_counts.append(nb_clusters)

print('On trouve entre ' + str(min(cluster_counts)) + ' clusters (valeurs élevées pour eps et min_samples)')
print('et ' + str(max(cluster_counts)) + ' clusters (valeurs faibles)')

# Problème : qd le nb de clusters diminue, le score de Davies-Bouldin augmente,
# ce qui est mauvais : cela signifie des clusters moins compacts et/ou moins séparés.
# Il y a donc un compromis à faire entre la qualité du clustering et son utilité métier

# Or on peut vérifier que pour 7 clusters le kmeans obtenait un score bien inférieur (donc meilleur)
# Encore une fois, sur ces données, le kmeans est à la fois + rapide et plus performant

# Reshape the scores and cluster counts for plotting
dbscan_scores = np.array(dbscan_scores).reshape(len(eps_values), len(min_samples_values))
cluster_counts = np.array(cluster_counts).reshape(len(eps_values), len(min_samples_values))

# Plot the results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(dbscan_scores, interpolation='nearest', cmap=plt.cm.autumn_r)
plt.title("DB Scores")
plt.xticks(np.arange(len(min_samples_values)), min_samples_values)
plt.yticks(np.arange(len(eps_values)), eps_values)
plt.xlabel("min_samples")
plt.ylabel("eps")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(cluster_counts, interpolation='nearest', cmap=plt.cm.autumn_r)
plt.title("Number of Clusters")
plt.xticks(np.arange(len(min_samples_values)), min_samples_values)
plt.yticks(np.arange(len(eps_values)), eps_values)
plt.xlabel("min_samples")
plt.ylabel("eps")
plt.colorbar()

plt.show()


In [ ]:
# custom gradual cmap,
# higher parameter values

color1 = (1.0, 1.0, 0.3)
color2 = (1.0, 0.1, 0.1)

num_segments = 256

gradient_colors = [color1, color2]
gradual_cmap = LinearSegmentedColormap.from_list("custom_cmap", gradient_colors, N=num_segments)

eps_values = [0.2, 0.3, 0.4]
min_samples_values = [5, 7, 9]

dbscan_scores = []
cluster_counts = []

# Perform manual grid search
for eps in eps_values:
    for min_samples in min_samples_values:
        dbscan = DBSCAN(eps=eps, min_samples=min_samples)
        labels = dbscan.fit_predict(rfm_log_scaled)
        score = davies_bouldin_score(rfm_log_scaled[features_RFM], labels)
        dbscan_scores.append(score)
        nb_clusters = len(set(labels))
        cluster_counts.append(nb_clusters)

print('On trouve entre ' + str(min(cluster_counts)) + ' clusters (valeurs élevées pour eps et min_samples)')
print('et ' + str(max(cluster_counts)) + ' clusters (valeurs faibles)')

dbscan_scores = np.array(dbscan_scores).reshape(len(eps_values), len(min_samples_values))
cluster_counts = np.array(cluster_counts).reshape(len(eps_values), len(min_samples_values))

# Plot the results
plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
plt.imshow(dbscan_scores, interpolation='nearest', cmap=gradual_cmap)
plt.title("DB Scores")
plt.xticks(np.arange(len(min_samples_values)), min_samples_values)
plt.yticks(np.arange(len(eps_values)), eps_values)
plt.xlabel("min_samples")
plt.ylabel("eps")
plt.colorbar()

plt.subplot(1, 2, 2)
plt.imshow(cluster_counts, interpolation='nearest', cmap=gradual_cmap)
plt.title("Number of Clusters")
plt.xticks(np.arange(len(min_samples_values)), min_samples_values)
plt.yticks(np.arange(len(eps_values)), eps_values)
plt.xlabel("min_samples")
plt.ylabel("eps")
plt.colorbar()

plt.show()

# mm conclusion que précédemment, pour 5 clusters le score du kmeans est bien meilleur.
# Et en plus kmeans va bcp + vite


## Conclusion features RFM


In [ ]:
# Lorsqu'on prend en compte uniquement ces 3 features,

# le nombre optimal de clusters que nous pouvons faire est de 4 clusters,
# ce qui est un bon début, mais encore trop peu pour une segmentation de qualité, selon les
# besoins d'Olis.
# Plutôt que de "forcer" un sous-partitionnement sub-optimal, nous allons ajouter une quatrième feature,
# la plus pertinente possible pour comprendre nos clients.

# Nous allons donc utiliser le score (moyen) de satisfaction indiqué par les clients, qui varie entre 1 et 5.

# Sur 3 dimensions, le k-means n'a pas juste été "plus performant" que les 2 autres algos ;
# il les a littéralement laissés dans la poussière ! (/ mis KO ?)


## Ajout du review_score, export


In [ ]:
rfms = rfm_log.copy()
rfms['satisfaction'] = rfm_complet['review_score']
rfms['datetime'] = rfm_complet['most_recent_order']

print(rfms.loc[rfms['nb_commandes'] > 0.7, :].shape[0]) # 0.693 = log(1+1)
# OK on n'a pas perdu nos 3%

quick_look(rfms)
rfms.describe()

# Il faut que je coupe ce notebook ici, ça a du sens, et c'est utile pour travailler
# (ça commence à être trop long de recharger tt le notebook qd c nécessaire)

rfms.to_csv('data/rfms.csv', sep=',', index=False)


## Annexes / tests


In [ ]:
# gaussian mixture models ?


## t-SNE


In [ ]:
# Le t-SNE ne permet pas d'interpréter les distances ou les tailles des clusters sur le graph obtenu,
# ce n'est donc pas du tout l'outil idéal pour une segmentation de clientèle.
# C juste pour l'essayer !

# Perform t-SNE
tsne = TSNE(n_components=2, perplexity=30, n_iter=300)
tsne_result = tsne.fit_transform(rfm_log_scaled[features_RFM])

# Create a DataFrame to hold the t-SNE results
df_tsne = pd.DataFrame(tsne_result, columns=["Dimension 1", "Dimension 2"])

# Visualize the t-SNE results
plt.figure(figsize=(8, 6))
plt.scatter(df_tsne["Dimension 1"], df_tsne["Dimension 2"])
plt.title("t-SNE Visualization")
plt.xlabel("Dimension 1")
plt.ylabel("Dimension 2")
plt.show()

# Wow ! De toute beauté !

# Après recherche, j'avais mal compris. Le t-SNE peut aider dans certains cas à détecter des clusters,
# mais ce n'est pas un outil de clusterisation à proprement parler. Il n'assigne pas de labels.
# C'est un outil de réduction dimentionnelle, et pour l'instant on n'en a mm pas besoin,
# on est "seulement" en 3D et on n'ira pas en dimensions très élevées.

# Ca reste une belle pomme, bleue comme une orange.
